# DEEE725 Speech Signal Processing Lab
### 2023 Spring, Kyungpook National University 
### Instructor: Gil-Jin Jang

## Lab 02 Draw spectrogram
2023/03/24
source: 

> https://hyunlee103.tistory.com/36

> https://dacon.io/en/competitions/official/235616/codeshare/1277

> https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
# import necessary pacakages
import sys
import numpy as np
import librosa
import librosa.display
from matplotlib import pyplot as plt

#from scipy.io import wavfile
from scipy import signal
from scipy.fft import fftshift

In [7]:
# parameters for signal analysis and drawing
#FIG_SIZE = (15,10)
Ts = 0.01   # 10 ms shift size
Tf = 0.02   # 20 ms frame size
#cmap_plot = plt.cm.bone # default colormap for spectrogram, gray
cmap_plot = plt.cm.bone_r # default colormap for spectrogram, gray
#cmap_plot = plt.cm.plasma 
#cmap_plot = plt.cm.inferno
Fs = 16000

#wavfile = 'kdigits0-2.wav'  # 0.6 - 1.1
wavfile = 'kdigits0-3.wav'  # 1.0 - 1.5

### load an example windows wav file

In [8]:
# load audio file with Librosa
x, Fs = librosa.load(wavfile, sr = Fs)
print(x,x.shape)

Ns = int(Fs*Ts)    # shift number of samples
Nf = int(Fs*Tf)    # frame number of samples
print('Fs = %d, Ns = %d, Nf = %d' % (Fs, Ns, Nf))

[-4.7912598e-03 -3.7231445e-03 -3.3569336e-03 ... -3.0517578e-05
 -8.8500977e-04  8.5449219e-04] (37952,)
Fs = 16000, Ns = 160, Nf = 320


### Waveform 시각화

In [12]:

#plt.figure(figsize=FIG_SIZE)
librosa.display.waveshow(x, sr=Fs)
# plt.plot(x) works as well 
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Waveform")

Text(0.5, 1.0, 'Waveform')

- 음성 부분만 tight하게 잘라낸다

In [24]:
x, Fs = librosa.load(wavfile, sr = Fs)
x = x[int(Fs*1.0):int(Fs*1.5)]

#plt.figure(figsize=FIG_SIZE)
librosa.display.waveshow(x, sr=Fs)
# plt.plot(x) works as well 
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Waveform")
plt.savefig("waveform.png")
plt.close()

### draw spectrogram

In [25]:
# draw spectrum by matplotlib
plt.figure()
[pxx,freq,t,cax] = plt.specgram(x,Fs=Fs,
        window=np.hamming(Ns*2),
        NFFT=Ns*2,noverlap=80,
        scale_by_freq=True,
        mode='psd',scale='dB',
        cmap=cmap_plot)

lab = 'input signal, PSD %.1f+/-%.1f'%(pxx[:].mean(),pxx[:].std())
plt.text(len(x)/Fs*0.05,Fs/8,lab)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram (dB)")
plt.savefig("spectrogram(db).png")

In [28]:
# draw spectrum by librosa
# https://dacon.io/en/competitions/official/235616/codeshare/1277

# STFT -> spectrogram
plt.figure()
stft = librosa.stft(x, n_fft=Nf, hop_length=Ns)
magnitude = np.abs(stft)
log_spectrogram = librosa.amplitude_to_db(magnitude)

plt.figure()
librosa.display.specshow(log_spectrogram, sr=Fs, hop_length=Ns)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram (dB)")
plt.savefig("spectrogram(db)2.png")
plt.close()

In [29]:
# draw spectrum by librosa, log scale in frequency
# https://librosa.org/doc/main/auto_examples/plot_display.html
plt.figure()
fig, ax = plt.subplots()
D_highres = librosa.stft(x, hop_length=Ns, n_fft=Nf)
S_db_hr = librosa.amplitude_to_db(np.abs(D_highres), ref=np.max)
img = librosa.display.specshow(S_db_hr, hop_length=Ns, x_axis='time', y_axis='log', ax=ax)
ax.set(title='Higher time and frequency resolution')
fig.colorbar(img, ax=ax, format="%+2.f dB")
plt.savefig("Higher time and frequency resolution.png")
plt.close()


In [31]:
# draw spectrum using scipy - not working
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html
plt.figure()
print(x.shape)
f, t, Sxx = signal.spectrogram(x, Fs, return_onesided=False)
plt.pcolormesh(t, fftshift(f), fftshift(Sxx, axes=0), shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()
plt.savefig("signal_spectrogram.png")
plt.close()

(8000,)


## 직접 그려본다

1. 20 ms 간격으로 나눈다

2. FFT 수행 

3. `imshow`로 그린다.

- 한 frame 그려보기 

In [32]:
# 0.2~0.22 부분을 추출한다. 
plt.figure()
y = x[int(Fs*0.2):int(Fs*0.22)]

In [34]:
Y = np.abs(np.fft.fft(y))
plt.plot(Y)
plt.savefig("1_Y.png")
plt.close()

In [35]:
# 0~PI+1 까지의 절대값
Y = np.abs(np.fft.fft(y))
Y = Y[:(len(Y)//2+1)]
plt.figure()
plt.plot(Y)
plt.savefig("2_Y.png")
plt.close()

In [36]:
# 0~PI+1 까지의 절대값
# log scale 로
Y = np.log(np.abs(np.fft.fft(y)))
Y = Y[:(len(Y)//2+1)]
plt.figure()
plt.plot(Y)
plt.savefig("3_Y.png")
plt.close()

In [37]:
# 0~PI+1 까지의 절대값
# log scale 로
plt.figure()
Y = np.log(np.abs(np.fft.fft(y)))
Y = Y[:(len(Y)//2+1)]
xticks=np.arange(len(Y))/len(Y)*Fs/2
plt.plot(xticks, Y)
plt.xlabel('frequency (Hz)')
plt.ylabel('log magnitude')
plt.savefig("4_log_manitude.png")
plt.close()

### Short-time Fourier transform

In [38]:
# Short-time Fourier transform
# 20ms 간격으로 나눈다
plt.figure()
T = len(x)   # number of samples
num_frames = T//Nf# 마지막 채워지지 않은 프레임은 버린다. 구현에 따라 zero-padding해서 사용 가능
hNo = Nf//2+1
X = np.zeros((hNo,num_frames))
for i in range(num_frames):
    y = np.fft.fft(x[(i*Nf):((i+1)*Nf)])
    y = y[:hNo]
    X[:,i] = np.abs(y)

In [39]:
plt.imshow(X, cmap=cmap_plot)
plt.savefig("5_stft.png")
plt.close()

In [40]:
plt.figure()
plt.imshow(X, cmap=cmap_plot, origin='lower')
plt.savefig("6_lower.png")
plt.close()

In [41]:
plt.figure()
specgram_axis = [0,float(len(x))/float(Fs),0,float(Fs)/2]
plt.imshow(X, cmap=cmap_plot, origin='lower', aspect='auto', extent=specgram_axis)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.savefig("7_time_axis_stft.png")
plt.close()

In [42]:
plt.figure()
# Short-time Fourier transform
# 20ms 간격으로 나눈다
T = len(x)   # number of samples
num_frames = T//Nf# 마지막 채워지지 않은 프레임은 버린다. 구현에 따라 zero-padding해서 사용 가능
hNo = Nf//2+1
X = np.zeros((hNo,num_frames))
for i in range(num_frames):
    y = np.fft.fft(x[(i*Nf):((i+1)*Nf)])
    y = y[:hNo]
    X[:,i] = np.abs(y)

# 상위 80% 정도만 scale 한다. imshow의 vmin vmax 이용 
vmax = np.max(X[:])
vmin = np.percentile(X[:], 91)
print(vmax, vmin)

specgram_axis = [0,float(len(x))/float(Fs),0,float(Fs)/2]
plt.imshow(X, cmap=cmap_plot, origin='lower', aspect='auto', extent=specgram_axis, vmax=vmax, vmin=vmin)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.savefig("8_20ms_stft.png")
plt.close()

13.12683388401886 0.7050746354016717


### Hamming windows

In [43]:
plt.figure()
T = 1024
sinusoid = np.sin(np.arange(T)/T*20*np.pi)
plt.plot(sinusoid)
plt.savefig("9_hanmming.png")
plt.close()

In [44]:
plt.figure()
# 0~PI+1 까지의 절대값
# log scale 로
Y = np.log(np.abs(np.fft.fft(sinusoid)))
Y = Y[:(len(Y)//2+1)]
xticks=np.arange(len(Y))/len(Y)*Fs/2
plt.plot(xticks, Y)
plt.xlabel('frequency (Hz)')
plt.ylabel('log magnitude')
plt.savefig("10_log_magnitude.png")
plt.close()

In [45]:
plt.figure()
win = librosa.filters.get_window('hamming', T, fftbins=True)
plt.plot(win)
plt.savefig("11_window.png")
plt.close()

In [46]:
plt.figure()
# 0~PI+1 까지의 절대값
# log scale 로
Y = np.log(np.abs(np.fft.fft(win*sinusoid)))
Y = Y[:(len(Y)//2+1)]
xticks=np.arange(len(Y))/len(Y)*Fs/2
plt.plot(xticks, Y)
plt.xlabel('frequency (Hz)')
plt.ylabel('log magnitude')
plt.savefig("12_log_magnitude.png")
plt.close()

### STFT에 hamming window 적용 

- 20 ms frame size, 10 ms shift size 적용, rectangular window

In [47]:
plt.figure()
# Short-time Fourier transform
# 10ms 간격, 20ms 분석 
T = len(x)   # number of samples
#num_frames = T//Nf# 마지막 채워지지 않은 프레임은 버린다. 구현에 따라 zero-padding해서 사용 가능
num_frames = (T-Nf)//Ns + 1 
hNo = Nf//2+1
X = np.zeros((hNo,num_frames))
win = librosa.filters.get_window('hamming', Nf, fftbins=True)
for i in range(num_frames):
    y = x[(i*Ns):(i*Ns+Nf)]
    y = np.fft.fft(y)
    y = y[:hNo]
    X[:,i] = np.abs(y)

# 상위 80% 정도만 scale 한다. imshow의 vmin vmax 이용 
vmax = np.max(X[:])
vmin = np.percentile(X[:], 91)
print(vmax, vmin)

specgram_axis = [0,float(len(x))/float(Fs),0,float(Fs)/2]
plt.imshow(X, cmap=cmap_plot, origin='lower', aspect='auto', extent=specgram_axis, vmax=vmax, vmin=vmin)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.savefig("13_80percent_spectogram.png")
plt.close()

13.767264121802409 0.6898897531907506


- 20 ms frame size, 10 ms shift size 적용, hamming window

In [48]:
plt.figure()
win = librosa.filters.get_window('hamming', Nf, fftbins=True)
plt.plot(win)
plt.savefig("14_librosa_hamming.png")
plt.close()

In [50]:
# Short-time Fourier transform
# 10ms 간격, 20ms 분석 
T = len(x)   # number of samples
#num_frames = T//Nf# 마지막 채워지지 않은 프레임은 버린다. 구현에 따라 zero-padding해서 사용 가능
num_frames = (T-Nf)//Ns + 1 
hNo = Nf//2+1
X = np.zeros((hNo,num_frames))
win = librosa.filters.get_window('hamming', Nf, fftbins=True)
for i in range(num_frames):
    y = x[(i*Ns):(i*Ns+Nf)]
    # hamming window
    y = win*y
    y = np.fft.fft(y)
    y = y[:hNo]
    X[:,i] = np.abs(y)

specgram_axis = [0,float(len(x))/float(Fs),0,float(Fs)/2]

# 상위 80%-90% 정도만 scale 한다. 출력 보면서 결정
vmax = np.max(X[:])
vmin = np.percentile(X[:], 80)
print(vmax, vmin)
plt.figure()
plt.imshow(X, cmap=cmap_plot, origin='lower', aspect='auto', extent=specgram_axis, vmax=vmax, vmin=vmin)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.savefig("16_80per_scale.png")
plt.close()

vmax = np.max(X[:])
vmin = np.percentile(X[:], 90)
print(vmax, vmin)
plt.figure()
plt.imshow(X, cmap=cmap_plot, origin='lower', aspect='auto', extent=specgram_axis, vmax=vmax, vmin=vmin)
plt.xlabel('time (seconds)')
plt.ylabel('frequency (Hz)')
plt.savefig("17_90per_scale.png")
plt.close()

7.731646815139541 0.08643033085681953
7.731646815139541 0.37138737706057884


## Resampling: Decimation, Upsampling, lowpass filter design

In [51]:
# scipy.signal.firwin(numtaps, cutoff, width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=None)[source]
# FIR filter design using the window method.




a = signal.firwin(51, 0.5, window='hamming')
plt.plot(a)

In [19]:
import numpy as np
import scipy
import librosa
import soundfile
import os

# change frequency
# fb : base frequency
# fc : change frequency
# signal : original signal(fb signal)
def change_freq(signal, fb, fc) :
    # calculate new signal length
    lenb = len(signal)
    lenc = int((lenb-1)*fc/fb) +1
    new_signal = np.zeros(lenc)
    for i in range(lenc) :
        # new_signal is generated linearly from 'num_sample' of signal.
        if i/fc*fb >= lenb-1 :
            new_signal[i] = signal[-1]
            break
        num_sample = int(i/fc*fb)
        low_sample = signal[num_sample]
        high_sample = signal[num_sample+1]
        new_signal[i] = (((i/fc)-(num_sample/fb))*high_sample + (((num_sample+1)/fb)-(i/fc))*low_sample)*fb

    return new_signal

# base wav frequency
filename = 'kdigits0-3'
fb = 16000
upsamples = [32000, 48000, 44100]
downsamples = [8000, 11025]
data, fb = librosa.load(f'{filename}.wav', sr = fb)

save_folder = 'hw2_result'
os.system('mkdir -p %s'%save_folder)
soundfile.write(os.path.join(save_folder,f'{filename}_16k.wav'), data, fb)

# upsampling
# change frequency -> low pass filter
for freq in upsamples :
    y = change_freq(data, fb, freq)
    low_filter = scipy.signal.firwin(51, fb//2, fs=freq, pass_zero='lowpass')
    y_f = scipy.signal.lfilter(low_filter, 1.0, y)
    soundfile.write(os.path.join(save_folder,f'{filename}_{freq//1000}k.wav'), y_f, freq)

# downsampling
# low pass filter -> change frequency
for freq in downsamples :
    low_filter = scipy.signal.firwin(51, freq//2, fs=fb, pass_zero='lowpass')
    y_f = scipy.signal.lfilter(low_filter, 1.0, data)
    y = change_freq(y_f, fb, freq)
    soundfile.write(os.path.join(save_folder,f'{filename}_{freq//1000}k.wav'), y, freq)


## End of Lab 02